In [8]:
import math
import csv
import numpy as np # type: ignore
import pandas as pd # type: ignore
from sklearn.cluster import KMeans # type: ignore
from sklearn.metrics.pairwise import cosine_similarity # type: ignore
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

ĐỌC VÀ XỬ LÝ DỮ LIỆU

In [2]:
data_ratings = pd.read_csv("data/ratings.dat", encoding="utf-8", sep='::', header=None, names=['userId','movieId','rating','tmp'], engine='python')
del data_ratings['tmp']

In [6]:
train_data, test_data = train_test_split(data_ratings, test_size=0.30, random_state=0)

In [7]:
# Chuyển dữ liệu sang định dạng ma trận
ratings_matrix = train_data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

# Tính ma trận tương tự cosin
similarity_matrix = cosine_similarity(ratings_matrix)

TÌM PHÂN CỤM TỐI ƯU VÀ PHÂN CỤM

In [ ]:
wcss = []
for c in range(1, 50):
    kmeans = KMeans(n_clusters=c, random_state=0).fit(similarity_matrix)
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(10, 5))
plt.plot(range(1, 50), wcss, marker="o")
plt.title("Elbow Method For Optimal k")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()
print(wcss)

In [5]:
# Phân cụm user sử dụng K-means
kmeans = KMeans(n_clusters=4, random_state=0).fit(similarity_matrix)

# Gán nhãn cho người dùng
user_clusters = pd.DataFrame({'userId': ratings_matrix.index, 'cluster': kmeans.labels_})

CHUẨN HOÁ DỮ LIỆU

In [6]:
# Chuyển đổi DataFrame sang từ điển
user_clusters_dict = user_clusters.set_index('userId').to_dict()['cluster']

In [7]:
# Thêm cột cluster vào DataFrame
train_data['cluster'] = train_data['userId'].map(dict(zip(ratings_matrix.index, kmeans.labels_)))

In [8]:
# Chuyển đổi ma trận tương tự thành dataframe
similarity_df = pd.DataFrame(similarity_matrix, index=ratings_matrix.index, columns=ratings_matrix.index)

# Chuyển dataframe sang long format
similarity_df = similarity_df.reset_index().melt(id_vars='userId', value_name='similarity', var_name='userId2')

# Bỏ các hàng có userId bằng userId2
similarity_df = similarity_df[similarity_df['userId'] != similarity_df['userId2']]

# Đổi tên cột
similarity_df = similarity_df.rename(columns={'userId': 'u1', 'userId2': 'u2'})

# Sắp xếp các giá trị theo u1, u2
similarity_df = similarity_df.sort_values(['u1', 'u2']).reset_index(drop=True)


In [9]:
# Chuyển similarity_df sang dictionary để tăng tốc độ truy vấn
similarity_dict = similarity_df.set_index('u1').groupby(level=0).apply(lambda x: x.set_index('u2')['similarity'].to_dict()).to_dict()

DỰ ĐOÁN ĐÁNH GIÁ BỘ PHIM

In [10]:
def predict_Rating_Nocluster(user, movie):
    # Lọc ra tất cả các đánh giá của bộ phim từ các người dùng khác, không bao gồm người dùng hiện tại
    user_ratings = train_data[(train_data['movieId'] == movie) & (train_data['userId'] != user)]

    # Lọc danh sách độ tương đồng của các user khác với user cần dự đoán
    similarities = similarity_dict.get(user, {})
    
    # Merge dữ liệu để lấy các similarity scores
    merge_df = user_ratings[user_ratings['userId'].isin(similarities.keys())].copy()
    merge_df['similarity'] = merge_df['userId'].map(similarities)

    # Tính weighted sum của ratings dựa trên similarity
    merge_df['weighted_rating'] = merge_df['rating'] * merge_df['similarity']

    # Tính tổng các trọng số và tổng trọng số rating
    sum_similarity = merge_df['similarity'].sum()
    sum_weighted_ratings = merge_df['weighted_rating'].sum()

    # Tính toán điểm rating dự đoán cuối cùng
    if sum_similarity > 0:
        predicted_rating = sum_weighted_ratings / sum_similarity
    else:
        predicted_rating = 0  # Nếu không có similarity nào, đặt giá trị mặc định là 0
    
    return predicted_rating


In [11]:
def predict_Rating_Cluster(user, movie):

    cluster = user_clusters_dict.get(user, {})

    # Lấy các user cùng cụm với user đầu vào và đã rate movie này
    user_ratings = train_data[(train_data['movieId'] == movie) & (train_data['cluster'] == cluster)]

    # Lọc danh sách độ tương đồng của các user khác với user cần dự đoán
    similarities = similarity_dict.get(user, {})
    
    # Merge dữ liệu để lấy các user cùng cụm và similarity scores
    merge_df = user_ratings[user_ratings['userId'].isin(similarities.keys())].copy()
    merge_df['similarity'] = merge_df['userId'].map(similarities)

    # Tính weighted sum của ratings dựa trên similarity
    merge_df['weighted_rating'] = merge_df['rating'] * merge_df['similarity']

    # Tính tổng các trọng số và tổng trọng số rating
    sum_similarity = merge_df['similarity'].sum()
    sum_weighted_ratings = merge_df['weighted_rating'].sum()

    # Tính toán điểm rating dự đoán cuối cùng
    if sum_similarity > 0:
        predicted_rating = sum_weighted_ratings / sum_similarity
    else:
        # Nếu không có similarity nào, sử dụng giá trị trung bình của cụm
        predicted_rating = 0
    
    return predicted_rating

THỜI GIAN DỰ ĐOÁN ĐÁNH GIÁ CHO BỘ PHIM

In [ ]:
def calculate_Time_Predict_Nocluster():
    predictions = []
    for index, row in test_data.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        start_time = time.time()
        predicted_rating = predict_Rating_Nocluster(user_id, movie_id)
        end_time = time.time()
        prediction_time = end_time - start_time
        predictions.append({'prediction_time': prediction_time})
        
    predictions_df = pd.DataFrame(predictions)
    mean_prediction_time = predictions_df['prediction_time'].mean()
    return mean_prediction_time

# Tính toán dự đoán rating cho tất cả các cặp user-movie
mean_prediction_time = calculate_Time_Predict_Nocluster()
print("mean_prediction_time: ",mean_prediction_time)


In [ ]:
def calculate_Time_Predict_Cluster():
    predictions = []
    for index, row in test_data.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        start_time = time.time()
        predicted_rating = predict_Rating_Cluster(user_id, movie_id)
        end_time = time.time()
        prediction_time = end_time - start_time
        predictions.append({'prediction_time': prediction_time})
        
    predictions_df = pd.DataFrame(predictions)
    mean_prediction_time = predictions_df['prediction_time'].mean()
    return mean_prediction_time

# Tính toán dự đoán rating cho tất cả các cặp user-movie
mean_prediction_time = calculate_Time_Predict_Cluster()
print("mean_prediction_time: ",mean_prediction_time)

TÍNH TOÁN MAE, RMSE CHO DỰ ĐOÁN ĐÁNH GIÁ BỘ PHIM

In [14]:
def MAE_RMSE_Predict_Nocluster(test_data):
    predictions = []
    actuals = []

    for index, row in test_data.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        actual_rating = row['rating']

        # Dự đoán rating dựa trên mô hình đã xây dựng
        predicted_rating = predict_Rating_Nocluster(user_id, movie_id)

        # Lưu lại giá trị dự đoán và giá trị thực tế
        predictions.append(predicted_rating)
        actuals.append(actual_rating)

    # Tạo DataFrame từ danh sách predictions và actuals
    results_df_predict_noclus = pd.DataFrame({
        'predicted': predictions,
        'actual': actuals
    })

    return results_df_predict_noclus

# Sử dụng hàm RMSE_Noclus để lấy DataFrame kết quả
results_df_predict_noclus = MAE_RMSE_Predict_Nocluster(test_data)

# Xuất DataFrame kết quả ra file Excel
results_df_predict_noclus.to_csv('results_noclus.csv', index=False, header=False)

In [15]:
def MAE_RMSE_Predict_Cluster(test_data):
    predictions = []
    actuals = []

    for index, row in test_data.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        actual_rating = row['rating']

        # Dự đoán rating dựa trên mô hình đã xây dựng
        predicted_rating = predict_Rating_Cluster(user_id, movie_id)

        # Lưu lại giá trị dự đoán và giá trị thực tế
        predictions.append(predicted_rating)
        actuals.append(actual_rating)

    # Tạo DataFrame từ danh sách predictions và actuals
    results_df_predict_clus = pd.DataFrame({
        'predicted': predictions,
        'actual': actuals
    })

    return results_df_predict_clus

# Sử dụng hàm RMSE_Noclus để lấy DataFrame kết quả
results_df_predict_clus = MAE_RMSE_Predict_Cluster(test_data)

# Xuất DataFrame kết quả ra file Excel
results_df_predict_clus.to_csv('results_clus.csv', index=False, header=False)

In [ ]:
results_df_predict_noclus = pd.read_csv("results_noclus.csv", encoding="utf-8", sep=',', header=None, names=['predicted','actual'], engine='python')

def evaluate_Predict_MAE_RMSE_Nocluster(results_df_predict_noclus):
    # Tính toán MSE và RMSE từ DataFrame
    mae = np.mean(np.abs(results_df_predict_noclus['predicted'] - results_df_predict_noclus['actual']))
    rmse = np.sqrt(np.mean((results_df_predict_noclus['predicted'] - results_df_predict_noclus['actual']) ** 2))
    return mae, rmse

# Tính toán MSE và RMSE từ DataFrame
test_mae, test_rmse = evaluate_Predict_MAE_RMSE_Nocluster(results_df_predict_noclus)
print(f'MAE của hàm predict_Rating_Nocluster là: {test_mae}')
print(f'RMSE của hàm predict_Rating_Nocluster là: {test_rmse}')

In [ ]:
results_df_predict_clus = pd.read_csv("results_clus.csv", encoding="utf-8", sep=',', header=None, names=['predicted','actual'], engine='python')

def evaluate_Predict_MAE_RMSE_Cluster(results_df_predict_clus):
    # Tính toán MSE và RMSE từ DataFrame
    mae = np.mean(np.abs(results_df_predict_clus['predicted'] - results_df_predict_clus['actual']))
    rmse = np.sqrt(np.mean((results_df_predict_clus['predicted'] - results_df_predict_clus['actual']) ** 2))
    return mae, rmse

# Tính toán MSE và RMSE từ DataFrame
test_mae, test_rmse = evaluate_Predict_MAE_RMSE_Cluster(results_df_predict_clus)

print(f'MAE của hàm predict_Rating_Cluster là: {test_mae}')
print(f'RMSE của hàm predict_Rating_Cluster là: {test_rmse}')